# Country Editor App  
Run the cell below and Voilá will render this notebook as a form.


In [1]:
# ONE-COLUMN COUNTRY-EDITOR FORM
# (copy this entire cell into editor_app.ipynb)

from pathlib import Path
import pandas as pd, ipywidgets as w
from IPython.display import display

# ---------- data helpers ----------
DATA = Path("country_data.csv")

def load():
    return pd.read_csv(DATA, dtype="string").fillna("")

def save(df):
    """Overwrite country_data.csv and keep a backup copy.

    Works on any Python ≥3.7 (no missing_ok keyword needed).
    """
    bak = DATA.with_suffix(".bak")
    if DATA.exists():                      # make backup only if file exists
        DATA.replace(bak)
    df.to_csv(DATA, index=False, encoding="utf-8")
    print(f"✔ saved {DATA.name} (backup → {bak.name})")

df = load()

# ---------- columns that default to Yes/No dropdown ----------
YESNO = {
    "Is use a criminal offence? [USE]",
    "Is use an administrative offence? [USE]",
    "Is possession a criminal offence? ",
    "Is possession an administrative offence? [POSSESSION]",
    "Mandatory Minimum Sentencing? ",
    "Penalty varies by drug? [USE]",
    "Penalty varies by drug? [POSSESSION]",
    "Penalty varies by drug? [SUPPLY]",
    # add more Yes/No columns here if needed
}

# ---------- widget builders ----------
widgets = {}

def wrap_label(text):
    "HTML label that wraps naturally."
    return w.HTML(f"<b>{text}</b>")

def make_input(col):
    if col in YESNO:
        # dropdown + optional free-text toggle
        dd     = w.Dropdown(options=["", "Yes", "No"], layout=w.Layout(width="160px"))
        txt    = w.Text(layout=w.Layout(width="460px"), visible=False)
        toggle = w.ToggleButton(icon="edit", tooltip="Free-text mode", layout=w.Layout(width="40px"))
        box    = w.HBox([dd, txt, toggle])

        def on_toggle(change):
            txt.visible = toggle.value
            dd.visible  = not toggle.value
        toggle.observe(on_toggle, names="value")

        # unified getters / setters
        box.get = lambda: txt.value if toggle.value else dd.value
        def _setter(v, _dd=dd, _txt=txt):
            if v in {"", "Yes", "No"}:
                _dd.value = v
                toggle.value = False
            else:
                _txt.value = v
                toggle.value = True
        box.set = _setter
        return box

    # default: multiline text area
    ta = w.Textarea(layout=w.Layout(width="660px", height="60px"))
    ta.get = lambda: ta.value
    ta.set = lambda v: ta.__setattr__("value", v)
    return ta

for col in df.columns:
    if col != "Country":
        widgets[col] = make_input(col)

# ---------- country dropdown ----------
country_dd = w.Dropdown(
    options=sorted(df["Country"].unique()) + ["<new country>"],
    description="Country",
    layout=w.Layout(width="320px")
)

# ---------- autofill on country change ----------
def autofill(change):
    name = change["new"]
    if name == "<new country>":
        for wdg in widgets.values():
            wdg.set("")
        return
    row = df[df["Country"] == name]
    for col, wdg in widgets.items():
        wdg.set("" if row.empty else row[col].iat[0])

country_dd.observe(autofill, names="value")

# ---------- save / update ----------
save_btn   = w.Button(description="Save / Update", button_style="success")
status_out = w.Output()

def on_save_clicked(_):
    name = country_dd.value
    if name == "<new country>":
        with status_out:
            status_out.clear_output()
            print("⚠ choose an existing country or add it to the dropdown list")
        return
    mask = df["Country"] == name
    if not mask.any():                       # add new row
        df.loc[len(df), "Country"] = name
        mask = df["Country"] == name
    for col, wdg in widgets.items():
        df.loc[mask, col] = wdg.get().strip()
    save(df)
    with status_out:
        status_out.clear_output()
        print("✅ saved row for", name)

save_btn.on_click(on_save_clicked)

# ---------- assemble single-column form ----------
rows  = [w.VBox([wrap_label(col), widgets[col]]) for col in df.columns if col != "Country"]
form  = w.VBox([country_dd] + rows + [save_btn, status_out],
               layout=w.Layout(align_items="flex-start"))

display(form)
autofill({"new": country_dd.value})   # populate initial selection
